# 1、获取大模型

In [1]:
#导入 dotenv 库的 load_dotenv 函数，用于加载环境变量文件（.env）中的配置
import dotenv
from langchain_openai import ChatOpenAI
import os

dotenv.load_dotenv()  #加载当前目录下的 .env 文件

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")  # 默认使用 gpt-3.5-turbo

# 直接提供问题，并调用llm
response = llm.invoke("什么是大模型？")
print(response)

PermissionDeniedError: Error code: 403 - {'error': {'message': '主账户可用余额不足，本次请求需要可用余额大于 0.01 元才能完成，您的可用余额为 -0.00876657 元(请求预冻结 0.00876657 元)，请充值后再使用', 'type': 'error'}}

# 2、使用提示词模板

In [2]:
from langchain_core.prompts import ChatPromptTemplate

# 需要注意的一点是，这里需要指明具体的role，在这里是system和用户
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档编写者"),
    ("user", "{input}")  # {input}为变量
])

# 我们可以把prompt和具体llm的调用和在一起。
chain = prompt | llm
message = chain.invoke({"input": "大模型中的LangChain是什么?"})
print(message)

# print(type(message))

content='LangChain是一个用于构建与大语言模型（如OpenAI的GPT系列、Google的BERT等）交互的应用程序的框架和工具集。它旨在简化和加速开发过程，使开发者能够更高效地创建、管理和集成基于语言模型的应用。LangChain的主要特点和功能包括：\n\n1. **组件化架构**：LangChain提供了一系列模块和组件，例如数据加载器、模型接口、链式调用等，这些组件可以根据需要组合使用，以实现复杂的应用逻辑。\n\n2. **多模型支持**：LangChain支持多种语言模型提供商，允许开发者选择最适合其应用需求的模型。\n\n3. **上下文管理**：LangChain能够管理对话和上下文，使得与用户的交互更加自然和连贯。\n\n4. **数据集成**：LangChain可以与各种数据源（如数据库、API、文件系统等）集成，以便从中提取和处理数据。\n\n5. **可扩展性**：开发者可以根据自己的需求扩展LangChain的功能，添加自定义组件和逻辑，以满足特定的业务场景。\n\n6. **社区和文档支持**：LangChain拥有活跃的社区和丰富的文档资料，方便开发者查找示例和进行技术交流。\n\n通过使用LangChain，开发者可以更容易地构建复杂的自然语言处理（NLP）应用，如聊天机器人、问答系统、内容生成工具等，从而加速产品的开发周期并提高项目的成功率。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 345, 'prompt_tokens': 29, 'total_tokens': 374, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_

# 3、 使用输出解析器

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

# 初始化模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档编写者。"),
    ("user", "{input}")
])

# 使用输出解析器
# output_parser = StrOutputParser()
output_parser = JsonOutputParser()

# 将其添加到上一个链中
# chain = prompt | llm
chain = prompt | llm | output_parser

# 调用它并提出同样的问题。答案是一个字符串，而不是ChatMessage
# chain.invoke({"input": "LangChain是什么?"})
chain.invoke({"input": "LangChain是什么? 用JSON格式回复，问题用question，回答用answer"})

{'question': 'LangChain是什么?',
 'answer': 'LangChain是一个用于构建基于大语言模型（LLM）应用程序的框架。它提供了多个模块和工具，使开发人员能够创建复杂的应用，如对话式代理、问答系统和文本生成工具。LangChain支持与多种LLM和数据源的连接，使得编写、调试和部署基于语言模型的应用变得更加高效。'}

# 4、使用向量存储

In [4]:
# 导入和使用 WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
        web_path="https://www.gov.cn/xinwen/2020-06/01/content_5516649.htm",
        bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="UCAP-CONTENT"))
    )
docs = loader.load()
# print(docs)

# 对于嵌入模型，这里通过 API调用
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 使用分割器分割文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
print(len(documents))
# 向量存储  embeddings 会将 documents 中的每个文本片段转换为向量，并将这些向量存储在 FAISS 向量数据库中
vector = FAISS.from_documents(documents, embeddings)

USER_AGENT environment variable not set, consider setting it to identify your requests.


248


# 5、RAG(检索增强生成)

In [5]:
from langchain_core.prompts import PromptTemplate

retriever = vector.as_retriever()
retriever.search_kwargs = {"k": 3}
docs = retriever.invoke("建设用地使用权是什么？")

# for i,doc in enumerate(docs):
#     print(f"⭐第{i+1}条规定：")
#     print(doc)

# 6.定义提示词模版
prompt_template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
{info}

用户问：
{question}

请用中文回答用户问题。
"""
# 7.得到提示词模版对象
template = PromptTemplate.from_template(prompt_template)

# 8.得到提示词对象
prompt = template.format(info=docs, question='建设用地使用权是什么？')

## 9. 调用LLM
response = llm.invoke(prompt)
print(response.content)

建设用地使用权是指依法设立的，允许使用特定土地进行建设的权利。根据相关规定，建设用地使用权可以在土地的地表、地上或者地下设立，并可以通过出让或划拨等方式取得。建设用地使用权人有权根据法律和合同的约定使用、转让、抵押等该土地，并且在建筑物、构筑物及其附属设施的所有权归属上，建设用地使用权人享有相应的权利。


# 6、使用Agent

In [6]:
from langchain.tools.retriever import create_retriever_tool

# 检索器工具
retriever_tool = create_retriever_tool(
    retriever,
    "CivilCodeRetriever",
    "搜索有关中华人民共和国民法典的信息。关于中华人民共和国民法典的任何问题，您必须使用此工具!",
)

tools = [retriever_tool]

from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# https://smith.langchain.com/hub
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 运行代理
agent_executor.invoke({"input": "建设用地使用权是什么"})

D:\developTools\miniconda3\envs\pyth310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `CivilCodeRetriever` with `{'query': '建设用地使用权'}`


第三百四十五条　建设用地使用权可以在土地的地表、地上或者地下分别设立。
第三百四十六条　设立建设用地使用权，应当符合节约资源、保护生态环境的要求，遵守法律、行政法规关于土地用途的规定，不得损害已经设立的用益物权。
第三百四十七条　设立建设用地使用权，可以采取出让或者划拨等方式。
工业、商业、旅游、娱乐和商品住宅等经营性用地以及同一土地有两个以上意向用地者的，应当采取招标、拍卖等公开竞价的方式出让。
严格限制以划拨方式设立建设用地使用权。
第三百四十八条　通过招标、拍卖、协议等出让方式设立建设用地使用权的，当事人应当采用书面形式订立建设用地使用权出让合同。
建设用地使用权出让合同一般包括下列条款：
（一）当事人的名称和住所；
（二）土地界址、面积等；
（三）建筑物、构筑物及其附属设施占用的空间；
（四）土地用途、规划条件；
（五）建设用地使用权期限；
（六）出让金等费用及其支付方式；
（七）解决争议的方法。
第三百四十九条　设立建设用地使用权的，应当向登记机构申请建设用地使用权登记。建设用地使用权自登记时设立。登记机构应当向建设用地使用权人发放权属证书。

第三百五十条　建设用地使用权人应当合理利用土地，不得改变土地用途；需要改变土地用途的，应当依法经有关行政主管部门批准。
第三百五十一条　建设用地使用权人应当依照法律规定以及合同约定支付出让金等费用。
第三百五十二条　建设用地使用权人建造的建筑物、构筑物及其附属设施的所有权属于建设用地使用权人，但是有相反证据证明的除外。
第三百五十三条　建设用地使用权人有权将建设用地使用权转让、互换、出资、赠与或者抵押，但是法律另有规定的除外。
第三百五十四条　建设用地使用权转让、互换、出资、赠与或者抵押的，当事人应当采用书面形式订立相应的合同。使用期限由当事人约定，但是不得超过建设用地使用权的剩余期限。
第三百五十五条　建设用地使用权转让、互换、出资或者赠与的，应当向登记机构申请变更登记。
第三百五十六条　建设用地使用权转让、互换、出资或者赠与的，附着于该土地上的建筑物、构筑物及其附属设施一并处分。
第三百五十七条　建筑物、构筑物

{'input': '建设用地使用权是什么',
 'output': '建设用地使用权是指由国家或集体出让或划拨给使用者在一定期间内对特定土地进行建设、开发和利用的权利。根据中华人民共和国民法典的相关规定，建设用地使用权的具体内容包括：\n\n1. **设立方式**：建设用地使用权可以通过出让或划拨的方式设立，且出让方式一般使用公开招标、拍卖等方式，特别是对于经营性用地。\n\n2. **合同内容**：建设用地使用权出让合同应包含当事人的信息、土地界址、面积、用途、使用权期限、出让金等费用及解决争议的方法。\n\n3. **登记**：建设用地使用权需向登记机构申请登记，登记后方可正式设立，并会发放权属证书。\n\n4. **合理利用和用途变更**：使用权人应合理利用土地，不得擅自改变土地用途，如需变更需经过相关行政部门的批准。\n\n5. **转让和抵押**：建设用地使用权人有权转让、出资、赠与或抵押该使用权，但需遵守法律规定。\n\n6. **使用权期限**：住宅建设用地使用权在期限届满后可以自动续期，而非住宅用地的续期则需依法办理。\n\n7. **补偿机制**：如因公共利益需要收回土地，建设用地使用权人有权获得相应的补偿。\n\n上述内容为建设用地使用权的基本信息和法律规定。'}